In [ ]:
!pip install datasets
!pip install transformers
!pip install faiss-cpu
!pip install langchain-community
!pip install -U langchain-huggingface

!pip install gradio
!pip install langchain
!pip install sentence-transformers

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

import gradio as gr
import pandas as pd



ERROR: Operation cancelled by user


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/HelpMate AI/FashionDB.csv')
df.head()

# Fill missing values and create a combined description
df['description'] = df['description'].fillna('')
df['combined'] = df['name'] + ' ' + df['description']

# Select relevant columns
data = df[['p_id', 'combined']]

# Initialize embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create a vector store
vector_store = FAISS.from_texts(data['combined'].tolist(), embedding_model)



In [ ]:
# Function to search for products based on user query
def search_products(query, n_results=5):
    query_embedding = embedding_model.embed_query(query)
    results = vector_store.similarity_search_by_vector(query_embedding, k=n_results)
    # Extract only the product descriptions from the results
    return [result.page_content for result in results]



In [ ]:
# results = search_products("red dress", n_results=5)
# for result in results:
#     print(result)

In [ ]:
# Create the Gradio interface
iface = gr.Interface(
    fn=search_products,
    inputs=gr.Textbox(lines=2, placeholder="Enter your query here..."),
    outputs=gr.Textbox(lines=7, label="Search Results"),
    title="Fashion Search",
    description="Enter a query to search for products.",
)

# Launch the interface
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://15fe37fc34df1e5f96.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
